In [63]:
import pandas as pd
import numpy as np
import random
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import OneHotEncoder

from sklearn.metrics import mean_squared_error, mean_absolute_error,r2_score
from sklearn.linear_model import LinearRegression
import time
import datetime

In [64]:
tripdata = pd.read_csv("../../dataset/final_bike_sharing.csv")

In [65]:
tripdata.head(3)

,index,tripduration,start_station_id,start_station_name,start_lat,start_lon,end_station_id,end_station_name,end_lat,end_lon,...,conditions,date,dist,birthyear,years_old,holiday,day,month,seasons,gender
0,259617,1.02,others,Manila & 1st,40.721651,-74.042884,others,Manila & 1st,40.721651,-74.042884,...,Clear,2018-09-05,1.783012e-12,1994,24,working_day,weekday,September,autumn,male
1,283363,1.02,others,Heights Elevator,40.748716,-74.040443,others,Heights Elevator,40.748716,-74.040443,...,cloudy_rain,2018-10-06,1.782506e-12,1994,24,working_day,weekend,October,autumn,male
2,169168,1.02,others,Van Vorst Park,40.718489,-74.047727,es_3185,City Hall,40.717732,-74.043845,...,Clear,2018-07-09,3.384331e-01,1963,55,working_day,weekday,July,summer,female


In [66]:
tripdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 315433 entries, 0 to 315432
Data columns (total 39 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   index               315433 non-null  int64  
 1   tripduration        315433 non-null  float64
 2   start_station_id    315433 non-null  object 
 3   start_station_name  315433 non-null  object 
 4   start_lat           315433 non-null  float64
 5   start_lon           315433 non-null  float64
 6   end_station_id      315433 non-null  object 
 7   end_station_name    315433 non-null  object 
 8   end_lat             315433 non-null  float64
 9   end_lon             315433 non-null  float64
 10  bikeid              315433 non-null  object 
 11  usertype            315433 non-null  object 
 12  hour                315433 non-null  int64  
 13  min                 315433 non-null  int64  
 14  temp                315433 non-null  float64
 15  feelslike           315433 non-nul

In [67]:
tripdata.columns

Index(['index', 'tripduration', 'start_station_id', 'start_station_name',
       'start_lat', 'start_lon', 'end_station_id', 'end_station_name',
       'end_lat', 'end_lon', 'bikeid', 'usertype', 'hour', 'min', 'temp',
       'feelslike', 'dew', 'humidity', 'precip', 'precipprob', 'snow',
       'snowdepth', 'windspeed', 'winddir', 'sealevelpressure', 'cloudcover',
       'visibility', 'solarradiation', 'uvindex', 'conditions', 'date', 'dist',
       'birthyear', 'years_old', 'holiday', 'day', 'month', 'seasons',
       'gender'],
      dtype='object')

## Selecting variables according to P values

In [68]:
selected_features = ['tripduration','start_lat','start_lon','end_lat','end_lon','usertype','hour',
                     'temp','feelslike','dew','snowdepth','winddir','sealevelpressure','visibility','solarradiation',
                     'conditions','dist','birthyear','holiday','day','month','gender']

In [69]:
df = tripdata[selected_features]

In [70]:
def split_dataset(df):
    X = df.drop('tripduration', axis=1)
    y = df['tripduration']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    print('Train', X_train.shape, y_train.shape)
    print('Test', X_test.shape, y_test.shape)
    return X_train, X_test, y_train, y_test

### AMputation with 5% NA values

In [71]:
## get the sample size such as 5%, 10%,20% etc
def get_sample_size(df, perc=1):
    return round(perc/100 * df.shape[0])

def get_an_index(index):
    li = []
    for i in index:
        li.append(i)
    return li   

def ampute_nan_values(X_train,y_train,col_name, perc):
    random.seed(100)
    index = get_an_index(X_train.index)
    sample_size = get_sample_size(X_train, perc)
    print("sample size is =", sample_size)
    selected_index = random.sample(index, sample_size)
    for i in selected_index:
        X_train.loc[i, col_name]= np.NaN
    return X_train, y_train

###====================####
def convert_to_dataframe(y_train):
    return pd.DataFrame(y_train)

def convert_to_series(y_train):
    return y_train.squeeze()

def ampute_nan_values_drop(X_train,y_train,col_name, perc):
    random.seed(100)
    index = get_an_index(X_train.index)
    sample_size = get_sample_size(X_train, perc)
    print("sample size is =", sample_size)
    selected_index = random.sample(index, sample_size)
    y_train = convert_to_dataframe(y_train)
    for i in selected_index:
        y_train.loc[i] = np.NaN
        X_train.loc[i,col_name]= np.NaN
    y_train = convert_to_series(y_train)
    return X_train, y_train

In [72]:
def sample_imputation_X(var,X_train,X_test):
    # extract a random sample
    random_sample_train = X_train[var].dropna().sample(X_train[var].isnull().sum(), random_state=0)
    random_sample_test = X_test[var].dropna().sample(X_test[var].isnull().sum(), random_state=0)
     # re-index the randomly extracted sample
    random_sample_train.index = X_train[X_train[var].isnull()].index
    random_sample_test.index = X_test[X_test[var].isnull()].index

    # replace the NA
    X_train.loc[X_train[var].isnull(), var] = random_sample_train
    X_test.loc[X_test[var].isnull(), var] = random_sample_test
    return X_train, X_test

In [73]:
def ampute_each_variables(variable, method,X_train,y_train,X_test,y_test):
    if method == "remove":
        X_train , y_train = ampute_nan_values_drop(X_train,y_train,variable,perc=5)
        print('Train set after Amputation', X_train.shape, y_train.shape)
        print("y_train Amputed value count of varaible ", y_train.isnull().sum())
        print("X_train Amputed value count of varaible ", X_train.isnull().sum()[variable])
        X_test , y_test = ampute_nan_values_drop(X_test,y_test,variable,perc=5)
        print('Test set after Amputation', X_test.shape, y_test.shape)
        print("y_test Amputed value count of varaible ", y_test.isnull().sum())
        print("X_test Amputed value count of varaible ", X_test.isnull().sum()[variable])
    else:
        X_train , y_train = ampute_nan_values(X_train,y_train,variable,perc=5)
        print('Train set after Amputation', X_train.shape, y_train.shape)
        print("X_train Amputed value count of varaible ", X_train.isnull().sum()[variable])
        X_test , y_test = ampute_nan_values(X_test,y_test,variable,perc=5)
        print('Test set after Amputation', X_test.shape, y_test.shape)
        print("X_test Amputed value count of varaible ", X_test.isnull().sum()[variable])
    start = datetime.datetime.now()
    if method == "remove":
        print("Simulate Imputation for NA")
        X_train.dropna(axis=0, inplace=True)
        y_train.dropna(inplace=True)
        print('Train after NA value removal Imputation', X_train.shape, y_train.shape)
        X_test.dropna(axis=0, inplace=True)
        y_test.dropna(inplace=True)
        print('Test after NA value removal Imputation', X_test.shape, y_test.shape)
    if method == "mode":
        print("Simulate Imputation for Mode")
        X_train.fillna(X_train[variable].mode()[0], inplace=True)
        print('Train after Mode Imputation', X_train.shape, y_train.shape)
        X_test.fillna(X_test[variable].mode()[0], inplace=True)
        print('Test after Mode Imputation', X_test.shape, y_test.shape)
    if method == "mean":
        print("Simulate Imputation for Mean")
        X_train.fillna(X_train[variable].mean(), inplace=True)
        print('Train after Mean Imputation', X_train.shape, y_train.shape)
        X_test.fillna(X_test[variable].mean(), inplace=True)
        print('Test after Mean Imputation', X_test.shape, y_test.shape)
    if method == "sample":
        print("Simple Imputation using sampling")
        X_train, X_test = sample_imputation_X(variable, X_train, X_test)
        
    end = datetime.datetime.now()
    comp_time = (end-start)
    print(f"Time taken simulate for variable {var} = ", comp_time)
    list_comp_time.append(comp_time)     
            
    return X_train , y_train, X_test , y_test

In [74]:
def one_hot_encoding(X_train, X_test):
    categorical_features = X_train.select_dtypes(include=['object']).columns.tolist()
    numerical_cols = X_train.select_dtypes(include=['int', 'float']).columns.tolist()
    ohe = OneHotEncoder(drop='first', sparse=False)
    X_train_encoded = ohe.fit_transform(X_train[categorical_features])
    X_test_encoded = ohe.fit_transform(X_test[categorical_features])
    X_train_new = np.hstack((X_train[numerical_cols].values, X_train_encoded))
    X_test_new = np.hstack((X_test[numerical_cols].values, X_test_encoded))
    return X_train_new, X_test_new

In [75]:
def model_evaluation(X_train,y_train):
    model_lr = LinearRegression(fit_intercept=True).fit(X_train, y_train) 
    return model_lr

In [76]:
features_for_amputation = ['start_lat','start_lon','end_lat','end_lon','usertype','hour',
                     'temp','feelslike','dew','snowdepth','winddir','sealevelpressure','visibility','solarradiation',
                     'conditions','dist','birthyear','holiday','day','month','gender']

## 1. Drop NA values 

In [77]:
model_metrics ={}
list_comp_time =[]
for var in features_for_amputation:
    print("Now running model is ....", var)
    X_train, X_test, y_train, y_test = split_dataset(df)
    X_train , y_train, X_test , y_test = ampute_each_variables(var,"remove",X_train,y_train,X_test,y_test)
    X_train, X_test = one_hot_encoding(X_train,X_test)
    model_reg = model_evaluation(X_train,y_train)
    y_pred = model_reg.predict(X_test)
    model_metrics[var] = {"MAE":mean_absolute_error(y_test, y_pred),"MSE":mean_squared_error(y_test, y_pred),
                          "RMSE":np.sqrt(mean_squared_error(y_test, y_pred)),"R2":r2_score(y_test, y_pred)}
    del X_train
    del X_test
    del y_train
    del y_test

Now running model is .... start_lat
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
sample size is = 12617
Train set after Amputation (252346, 21) (252346,)
y_train Amputed value count of varaible  12617
X_train Amputed value count of varaible  12617
sample size is = 3154
Test set after Amputation (63087, 21) (63087,)
y_test Amputed value count of varaible  3154
X_test Amputed value count of varaible  3154
Simulate Imputation for NA
Train after NA value removal Imputation (239729, 21) (239729,)
Test after NA value removal Imputation (59933, 21) (59933,)
Time taken simulate for variable start_lat =  0:00:00.441636
Now running model is .... start_lon
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
sample size is = 12617
Train set after Amputation (252346, 21) (252346,)
y_train Amputed value count of varaible  12617
X_train Amputed value count of varaible  12617
sample size is = 3154
Test set after Amputation (63087, 21) (63087,)
y_test Amputed value count of varaible  3154


Train set after Amputation (252346, 21) (252346,)
y_train Amputed value count of varaible  12617
X_train Amputed value count of varaible  12617
sample size is = 3154
Test set after Amputation (63087, 21) (63087,)
y_test Amputed value count of varaible  3154
X_test Amputed value count of varaible  3154
Simulate Imputation for NA
Train after NA value removal Imputation (239729, 21) (239729,)
Test after NA value removal Imputation (59933, 21) (59933,)
Time taken simulate for variable solarradiation =  0:00:00.376290
Now running model is .... conditions
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
sample size is = 12617
Train set after Amputation (252346, 21) (252346,)
y_train Amputed value count of varaible  12617
X_train Amputed value count of varaible  12617
sample size is = 3154
Test set after Amputation (63087, 21) (63087,)
y_test Amputed value count of varaible  3154
X_test Amputed value count of varaible  3154
Simulate Imputation for NA
Train after NA value removal Imputat

In [78]:
print("averge time to run drop is : ", 0.3785755, "seconds")
list_comp_time

averge time to run drop is :  344758.7619 seconds


[datetime.timedelta(microseconds=441636),
 datetime.timedelta(microseconds=391956),
 datetime.timedelta(microseconds=338912),
 datetime.timedelta(microseconds=366214),
 datetime.timedelta(microseconds=351981),
 datetime.timedelta(microseconds=302512),
 datetime.timedelta(microseconds=439293),
 datetime.timedelta(microseconds=394871),
 datetime.timedelta(microseconds=344381),
 datetime.timedelta(microseconds=344480),
 datetime.timedelta(microseconds=423283),
 datetime.timedelta(microseconds=279994),
 datetime.timedelta(microseconds=326562),
 datetime.timedelta(microseconds=376290),
 datetime.timedelta(microseconds=364758),
 datetime.timedelta(microseconds=423026),
 datetime.timedelta(microseconds=370020),
 datetime.timedelta(microseconds=420587),
 datetime.timedelta(microseconds=352360),
 datetime.timedelta(microseconds=382889),
 datetime.timedelta(microseconds=514081)]

In [79]:
model_metrics

{'start_lat': {'MAE': 1.57221692977683,
  'MSE': 5.003044094304209,
  'RMSE': 2.2367485541079954,
  'R2': 0.5261814122364713},
 'start_lon': {'MAE': 1.57221692977683,
  'MSE': 5.003044094304209,
  'RMSE': 2.2367485541079954,
  'R2': 0.5261814122364713},
 'end_lat': {'MAE': 1.57221692977683,
  'MSE': 5.003044094304209,
  'RMSE': 2.2367485541079954,
  'R2': 0.5261814122364713},
 'end_lon': {'MAE': 1.57221692977683,
  'MSE': 5.003044094304209,
  'RMSE': 2.2367485541079954,
  'R2': 0.5261814122364713},
 'usertype': {'MAE': 1.57221692977683,
  'MSE': 5.003044094304209,
  'RMSE': 2.2367485541079954,
  'R2': 0.5261814122364713},
 'hour': {'MAE': 1.57221692977683,
  'MSE': 5.003044094304209,
  'RMSE': 2.2367485541079954,
  'R2': 0.5261814122364713},
 'temp': {'MAE': 1.57221692977683,
  'MSE': 5.003044094304209,
  'RMSE': 2.2367485541079954,
  'R2': 0.5261814122364713},
 'feelslike': {'MAE': 1.57221692977683,
  'MSE': 5.003044094304209,
  'RMSE': 2.2367485541079954,
  'R2': 0.5261814122364713},

### Mode Replacement

In [80]:
model_metrics_mode ={}
list_comp_time =[]
for var in features_for_amputation:
    print("Now model is running for varaible ====", var)
    X_train, X_test, y_train, y_test = split_dataset(df)
    X_train , y_train, X_test , y_test = ampute_each_variables(var,"mode",X_train,y_train,X_test,y_test)
    X_train, X_test = one_hot_encoding(X_train,X_test)
    model_reg = model_evaluation(X_train,y_train)
    y_pred = model_reg.predict(X_test)
    model_metrics_mode[var] = {"MAE":mean_absolute_error(y_test, y_pred),"MSE":mean_squared_error(y_test, y_pred),
                          "RMSE":np.sqrt(mean_squared_error(y_test, y_pred)),"R2":r2_score(y_test, y_pred)}
    
    del X_train
    del X_test
    del y_train
    del y_test

Now model is running for varaible ==== start_lat
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
sample size is = 12617
Train set after Amputation (252346, 21) (252346,)
X_train Amputed value count of varaible  12617
sample size is = 3154
Test set after Amputation (63087, 21) (63087,)
X_test Amputed value count of varaible  3154
Simulate Imputation for Mode
Train after Mode Imputation (252346, 21) (252346,)
Test after Mode Imputation (63087, 21) (63087,)
Time taken simulate for variable start_lat =  0:00:00.407161
Now model is running for varaible ==== start_lon
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
sample size is = 12617
Train set after Amputation (252346, 21) (252346,)
X_train Amputed value count of varaible  12617
sample size is = 3154
Test set after Amputation (63087, 21) (63087,)
X_test Amputed value count of varaible  3154
Simulate Imputation for Mode
Train after Mode Imputation (252346, 21) (252346,)
Test after Mode Imputation (63087, 21) (63087,)
Time ta

Train after Mode Imputation (252346, 21) (252346,)
Test after Mode Imputation (63087, 21) (63087,)
Time taken simulate for variable dist =  0:00:00.323462
Now model is running for varaible ==== birthyear
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
sample size is = 12617
Train set after Amputation (252346, 21) (252346,)
X_train Amputed value count of varaible  12617
sample size is = 3154
Test set after Amputation (63087, 21) (63087,)
X_test Amputed value count of varaible  3154
Simulate Imputation for Mode
Train after Mode Imputation (252346, 21) (252346,)
Test after Mode Imputation (63087, 21) (63087,)
Time taken simulate for variable birthyear =  0:00:00.392881
Now model is running for varaible ==== holiday
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
sample size is = 12617
Train set after Amputation (252346, 21) (252346,)
X_train Amputed value count of varaible  12617
sample size is = 3154
Test set after Amputation (63087, 21) (63087,)
X_test Amputed value count 

In [82]:
print("averge time to run mode is : ", 0.3316852, "seconds")
list_comp_time

averge time to run mode is :  0.31559 seconds


[datetime.timedelta(microseconds=407161),
 datetime.timedelta(microseconds=311945),
 datetime.timedelta(microseconds=291452),
 datetime.timedelta(microseconds=286079),
 datetime.timedelta(microseconds=336591),
 datetime.timedelta(microseconds=381810),
 datetime.timedelta(microseconds=282192),
 datetime.timedelta(microseconds=366178),
 datetime.timedelta(microseconds=338345),
 datetime.timedelta(microseconds=336465),
 datetime.timedelta(microseconds=369306),
 datetime.timedelta(microseconds=326344),
 datetime.timedelta(microseconds=317946),
 datetime.timedelta(microseconds=282873),
 datetime.timedelta(microseconds=336158),
 datetime.timedelta(microseconds=323462),
 datetime.timedelta(microseconds=392881),
 datetime.timedelta(microseconds=286882),
 datetime.timedelta(microseconds=270330),
 datetime.timedelta(microseconds=349477),
 datetime.timedelta(microseconds=371514)]

In [83]:
model_metrics_mode

{'start_lat': {'MAE': 1.5722905984580589,
  'MSE': 4.9998369146695305,
  'RMSE': 2.236031510213917,
  'R2': 0.527214559240752},
 'start_lon': {'MAE': 1.5723721513544735,
  'MSE': 4.9999780986014235,
  'RMSE': 2.236063080192825,
  'R2': 0.5272012088638082},
 'end_lat': {'MAE': 1.5723099398113074,
  'MSE': 4.999841862808739,
  'RMSE': 2.2360326166692515,
  'R2': 0.5272140913438552},
 'end_lon': {'MAE': 1.5726237816482134,
  'MSE': 5.0003744961263275,
  'RMSE': 2.2361517158114133,
  'R2': 0.5271637254455035},
 'usertype': {'MAE': 1.5738518028291473,
  'MSE': 5.011245350651059,
  'RMSE': 2.238581102093703,
  'R2': 0.5261357755672135},
 'hour': {'MAE': 1.5730073666560653,
  'MSE': 5.0032957554188116,
  'RMSE': 2.2368048094142705,
  'R2': 0.5268874906631016},
 'temp': {'MAE': 1.5724724024106442,
  'MSE': 5.000529427936508,
  'RMSE': 2.236186358051696,
  'R2': 0.5271490750668184},
 'feelslike': {'MAE': 1.57247219039644,
  'MSE': 5.000336215551916,
  'RMSE': 2.236143156318914,
  'R2': 0.527167

### Mean Replacement

In [84]:
variables_for_mean_amputation = ['start_lat','start_lon','end_lat','end_lon','hour',
                                 'temp','feelslike','dew','snowdepth','winddir','visibility','sealevelpressure',
                                 'solarradiation','dist','birthyear']

In [85]:
model_metrics_mean ={}
list_comp_time =[]
for var in variables_for_mean_amputation:
    print("Now model is running for varaible ====", var)
    X_train, X_test, y_train, y_test = split_dataset(df)
    X_train , y_train, X_test , y_test = ampute_each_variables(var,"mean",X_train,y_train,X_test,y_test)
    X_train, X_test = one_hot_encoding(X_train,X_test)
    model_reg = model_evaluation(X_train,y_train)
    y_pred = model_reg.predict(X_test)
    model_metrics_mean[var] = {"MAE":mean_absolute_error(y_test, y_pred),"MSE":mean_squared_error(y_test, y_pred),
                          "RMSE":np.sqrt(mean_squared_error(y_test, y_pred)),"R2":r2_score(y_test, y_pred)}
    del X_train
    del X_test
    del y_train
    del y_test

Now model is running for varaible ==== start_lat
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
sample size is = 12617
Train set after Amputation (252346, 21) (252346,)
X_train Amputed value count of varaible  12617
sample size is = 3154
Test set after Amputation (63087, 21) (63087,)
X_test Amputed value count of varaible  3154
Simulate Imputation for Mean
Train after Mean Imputation (252346, 21) (252346,)
Test after Mean Imputation (63087, 21) (63087,)
Time taken simulate for variable start_lat =  0:00:00.319854
Now model is running for varaible ==== start_lon
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
sample size is = 12617
Train set after Amputation (252346, 21) (252346,)
X_train Amputed value count of varaible  12617
sample size is = 3154
Test set after Amputation (63087, 21) (63087,)
X_test Amputed value count of varaible  3154
Simulate Imputation for Mean
Train after Mean Imputation (252346, 21) (252346,)
Test after Mean Imputation (63087, 21) (63087,)
Time ta

In [86]:
print("averge time to run mean is : ", 0.314840, "seconds")
list_comp_time

averge time to run mean is :  0.31484 seconds


[datetime.timedelta(microseconds=319854),
 datetime.timedelta(microseconds=272747),
 datetime.timedelta(microseconds=307796),
 datetime.timedelta(microseconds=263675),
 datetime.timedelta(microseconds=331007),
 datetime.timedelta(microseconds=309425),
 datetime.timedelta(microseconds=261747),
 datetime.timedelta(microseconds=283864),
 datetime.timedelta(microseconds=327981),
 datetime.timedelta(microseconds=328215),
 datetime.timedelta(microseconds=265528),
 datetime.timedelta(microseconds=236451),
 datetime.timedelta(microseconds=302709),
 datetime.timedelta(microseconds=346145),
 datetime.timedelta(microseconds=332710)]

In [87]:
model_metrics_mean

{'start_lat': {'MAE': 1.5722749771969022,
  'MSE': 4.99976190726139,
  'RMSE': 2.2360147377111335,
  'R2': 0.527221651954199},
 'start_lon': {'MAE': 1.572408283143903,
  'MSE': 5.000041189486412,
  'RMSE': 2.236077187729979,
  'R2': 0.5271952429788451},
 'end_lat': {'MAE': 1.5722757377130143,
  'MSE': 4.999713074161806,
  'RMSE': 2.2360038180114556,
  'R2': 0.5272262696205158},
 'end_lon': {'MAE': 1.5725788061775683,
  'MSE': 5.000213078750087,
  'RMSE': 2.23611562284916,
  'R2': 0.5271789891004336},
 'hour': {'MAE': 1.572577536147564,
  'MSE': 5.001060542520806,
  'RMSE': 2.236305109442986,
  'R2': 0.5270988527801486},
 'temp': {'MAE': 1.5724445228548793,
  'MSE': 5.00038836530324,
  'RMSE': 2.236154816935366,
  'R2': 0.5271624139737433},
 'feelslike': {'MAE': 1.572456742799065,
  'MSE': 5.000289891402653,
  'RMSE': 2.2361327982484966,
  'R2': 0.5271717256827624},
 'dew': {'MAE': 1.5722703842567702,
  'MSE': 4.999404189211033,
  'RMSE': 2.235934746188053,
  'R2': 0.5272554778347212},


### Sampling Replacement

In [88]:
model_metrics_sample ={}
list_comp_time =[]
for var in features_for_amputation:
    print("Now model is running for varaible ====", var)
    X_train, X_test, y_train, y_test = split_dataset(df)
    X_train , y_train, X_test , y_test = ampute_each_variables(var,"sample",X_train,y_train,X_test,y_test)
    X_train, X_test = one_hot_encoding(X_train,X_test)
    model_reg = model_evaluation(X_train,y_train)
    y_pred = model_reg.predict(X_test)
    model_metrics_sample[var] = {"MAE":mean_absolute_error(y_test, y_pred),"MSE":mean_squared_error(y_test, y_pred),
                          "RMSE":np.sqrt(mean_squared_error(y_test, y_pred)),"R2":r2_score(y_test, y_pred)}
    del X_train
    del X_test
    del y_train
    del y_test

Now model is running for varaible ==== start_lat
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
sample size is = 12617
Train set after Amputation (252346, 21) (252346,)
X_train Amputed value count of varaible  12617
sample size is = 3154
Test set after Amputation (63087, 21) (63087,)
X_test Amputed value count of varaible  3154
Simple Imputation using sampling
Time taken simulate for variable start_lat =  0:00:00.046560
Now model is running for varaible ==== start_lon
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
sample size is = 12617
Train set after Amputation (252346, 21) (252346,)
X_train Amputed value count of varaible  12617
sample size is = 3154
Test set after Amputation (63087, 21) (63087,)
X_test Amputed value count of varaible  3154
Simple Imputation using sampling
Time taken simulate for variable start_lon =  0:00:00.024991
Now model is running for varaible ==== end_lat
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
sample size is = 12617
Train set a

sample size is = 12617
Train set after Amputation (252346, 21) (252346,)
X_train Amputed value count of varaible  12617
sample size is = 3154
Test set after Amputation (63087, 21) (63087,)
X_test Amputed value count of varaible  3154
Simple Imputation using sampling
Time taken simulate for variable month =  0:00:00.067525
Now model is running for varaible ==== gender
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
sample size is = 12617
Train set after Amputation (252346, 21) (252346,)
X_train Amputed value count of varaible  12617
sample size is = 3154
Test set after Amputation (63087, 21) (63087,)
X_test Amputed value count of varaible  3154
Simple Imputation using sampling
Time taken simulate for variable gender =  0:00:00.063732


In [89]:
print("averge time to run sample is : ", 0.454537,"seconds")
list_comp_time

averge time to run sample is :  0.47279 seconds


[datetime.timedelta(microseconds=46560),
 datetime.timedelta(microseconds=24991),
 datetime.timedelta(microseconds=22061),
 datetime.timedelta(microseconds=26934),
 datetime.timedelta(microseconds=83718),
 datetime.timedelta(microseconds=62720),
 datetime.timedelta(microseconds=38993),
 datetime.timedelta(microseconds=25579),
 datetime.timedelta(microseconds=24990),
 datetime.timedelta(microseconds=24035),
 datetime.timedelta(microseconds=23000),
 datetime.timedelta(microseconds=21676),
 datetime.timedelta(microseconds=22003),
 datetime.timedelta(microseconds=34693),
 datetime.timedelta(microseconds=76827),
 datetime.timedelta(microseconds=40593),
 datetime.timedelta(microseconds=46656),
 datetime.timedelta(microseconds=89446),
 datetime.timedelta(microseconds=87796),
 datetime.timedelta(microseconds=67525),
 datetime.timedelta(microseconds=63732)]

In [90]:
model_metrics_sample

{'start_lat': {'MAE': 1.5722497081781581,
  'MSE': 4.999946099221386,
  'RMSE': 2.2360559248868053,
  'R2': 0.5272042347307021},
 'start_lon': {'MAE': 1.5726021562608021,
  'MSE': 5.000488891245084,
  'RMSE': 2.2361772942334164,
  'R2': 0.5271529082233491},
 'end_lat': {'MAE': 1.5722990098325715,
  'MSE': 5.00010224805974,
  'RMSE': 2.2360908407441187,
  'R2': 0.5271894692696231},
 'end_lon': {'MAE': 1.5726064637800512,
  'MSE': 5.000956474305045,
  'RMSE': 2.2362818414289927,
  'R2': 0.5271086934885751},
 'usertype': {'MAE': 1.5751892105740941,
  'MSE': 5.018800988344268,
  'RMSE': 2.2402680617158897,
  'R2': 0.5254213131641459},
 'hour': {'MAE': 1.572931757129256,
  'MSE': 5.002814394561246,
  'RMSE': 2.236697206722726,
  'R2': 0.5269330082287927},
 'temp': {'MAE': 1.5724927645228082,
  'MSE': 5.000577962271786,
  'RMSE': 2.2361972100581347,
  'R2': 0.5271444856517059},
 'feelslike': {'MAE': 1.57242716873781,
  'MSE': 5.000206470697878,
  'RMSE': 2.2361141452747617,
  'R2': 0.5271796